In [1]:
import numpy as np
import torch
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

import src
from src.bert import training
from src.bert.dataset import PBertDataset
from src.bert.dataset.strategies import MLMin1PopIdeol

In [2]:
EXCLUDE_CODERS: list[str] = []
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

MODEL = "deepset/gbert-large"
BATCH_SIZE = 8
N_EPOCHS = 3
LR = 0.000009
WEIGHT_DECAY = 0.01

THRESHOLDS = {0: 0.415961, 1: 0.295400, 2: 0.429109, 3: 0.302714}

In [3]:
train = PBertDataset.from_disk(
    path=src.PATH / "data/labeled_data/train.csv.zip",
    label_strategy=MLMin1PopIdeol(),
    exclude_coders=EXCLUDE_CODERS,
)

test = PBertDataset.from_disk(
    path=src.PATH / "data/labeled_data/test.csv.zip",
    label_strategy=MLMin1PopIdeol(),
    exclude_coders=EXCLUDE_CODERS,
)

In [4]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)
collate_fn = train.create_collate_fn(tokenizer)

train_loader = DataLoader(train, collate_fn=collate_fn, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test, collate_fn=collate_fn, batch_size=64, shuffle=False)

In [5]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=train.num_labels).to(
    DEVICE
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at deepset/gbert-large and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=LR,
    amsgrad=False,
    weight_decay=WEIGHT_DECAY,
)

lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=20,
    eta_min=LR / 10,
)

for epoch in range(1, N_EPOCHS + 1):
    train_loss = training.train_epoch(model, train_loader, optimizer, lr_scheduler, clip=5)
    eval_loss, score, _ = training.eval_epoch(model, test_loader)
    print(f"{epoch=} {train_loss=:.4f} {eval_loss=:.4f} {score=:.4f}")

epoch=1 train_loss=284.3637 eval_loss=7.3026 score=0.5842


epoch=2 train_loss=188.7223 eval_loss=6.9862 score=0.7222


epoch=3 train_loss=133.5404 eval_loss=6.7450 score=0.7381


In [7]:
def apply_thresh(y_proba, thresholds: dict):
    y_proba = y_proba.copy()
    for dim, thresh in thresholds.items():
        y_proba[:, dim] = np.where(y_proba[:, dim] > thresh, 1, 0)
    return y_proba

In [8]:
with torch.inference_mode():
    y_true = []
    y_pred = []
    for batch in test_loader:
        encodings = batch["encodings"]
        encodings = encodings.to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        out = model(**encodings)
        preds = torch.nn.functional.sigmoid(out.logits)
        y_true.extend(batch["labels"].numpy())
        y_pred.extend(preds.cpu().numpy())
    y_pred_05 = np.where(np.array(y_pred) > 0.5, 1, 0)
    y_pred_thresh = apply_thresh(np.array(y_pred), THRESHOLDS)
    y_true = np.array(y_true)

In [9]:
print(classification_report(y_true, y_pred_05, target_names=["elite", "pplcentr", "left", "right"]))

              precision    recall  f1-score   support

       elite       0.82      0.87      0.85       648
    pplcentr       0.68      0.74      0.71       322
        left       0.73      0.81      0.77       279
       right       0.71      0.57      0.63       155

   micro avg       0.76      0.80      0.78      1404
   macro avg       0.73      0.75      0.74      1404
weighted avg       0.76      0.80      0.77      1404
 samples avg       0.41      0.40      0.40      1404



/gpfs/bwfor/home/st/st_st/st_ac138201/bert_populism/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/bwfor/home/st/st_st/st_ac138201/bert_populism/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
print(
    classification_report(
        y_true, y_pred_thresh, target_names=["elite", "pplcentr", "left", "right"]
    )
)

              precision    recall  f1-score   support

       elite       0.81      0.89      0.85       648
    pplcentr       0.64      0.82      0.72       322
        left       0.71      0.81      0.76       279
       right       0.61      0.68      0.64       155

   micro avg       0.72      0.84      0.78      1404
   macro avg       0.69      0.80      0.74      1404
weighted avg       0.73      0.84      0.78      1404
 samples avg       0.42      0.42      0.41      1404



/gpfs/bwfor/home/st/st_st/st_ac138201/bert_populism/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/bwfor/home/st/st_st/st_ac138201/bert_populism/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
model.save_pretrained(src.PATH / "results/popbert_model")